In [1]:
%display latex
import pylie, MatrixOrder, Cont
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint

In [3]:
class Differential_Polynomial:
    def __init__ (self, e, context):        
        self._orig = e     
        self._init(e, context)
      
    def _init(self, e, context):  
        res = []
        if pylie.is_derivative(e):
            res = [DTerm(self._orig)]
        else:
            for operand in self._orig.operands():
                dterm = DTerm(operand)
                c, d  = dterm._coeff, dterm._d
                inserted = False
                for r in res:
                    if r._d == d:
                        r._coeff += c
                        inserted  = True
                        # not very elegant but enough in first order
                        break 
                if not inserted :
                    res.append (dterm)        
        if len(res) > 1:
            res=sorted(res, 
                       key=functools.cmp_to_key(
                           lambda item1, item2: 
                                MatrixOrder.sorter (item1._d, item2._d , context._weight, 
                                                    tuple(context._dependent), 
                                                    tuple(context._independent)
                                                   )
                       )
                      )
        self._p = res
        self.normalize()
        
    def Lterm (self):
        return self._p[0].term()
        
    def Lder (self):
        return self._p[0]._d
    
    def Lcoeff(self):
        return self._p[0]
    
    def terms (self):
        for p in self._p:
            yield p.term()
            
    def derivatives (self):
        for p in self._p:
            yield p._d
            
    def coefficients (self):
        for p in self._p:
            yield p._coeff
    def is_monic (self):
        if self._p:
            return self._p[0].is_monic()
        return True
    def normalize (self):
        self._p = [ DTerm((_._coeff / self._p[0]._coeff) * _._d) for _ in self._p]
    def __str__ (self):
        return str(self._orig)

ERROR:root:No traceback has been produced, nothing to debug.


TypeError: reduce() of empty sequence with no initial value

In [ ]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    # XXX except only function names in context list
    assert e2.is_monic()
    for t, d, c, in zip (e1.terms(), e1.derivatives(), e1.coefficients()):
        ld = e2.Lder()
        def _order (der):
            if der != 1:
                return pylie.order_of_derivative(der)
            else :
                return [0]*len(context._independent)
        e2_order = _order (ld)
        e1_order = _order (d)
        dif = []
        for a, b in zip (e1_order, e2_order):
            dif += [a - b]
        try:
            if ld.operator().function() != d.operator().function():
                continue
        except AttributeError:
            continue            
        if all (map (lambda h: h == 0, dif)) :
            # XXX: do we need that in homogenous environmeent?
            inter = sum(_ for _ in e1.terms()) - c * sum (_ for _ in e2.terms())
            dp = Differential_Polynomial(inter.simplify_full().expand(), context)
            assert dp.is_monic()
            return dp
        elif  all (map (lambda h: h >= 0, dif)):
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            # get variables which shuld be differentiate
            variables_to_diff = []
            for i in range (len(context._independent)):
                if dif [i] != 0:
                    variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))
            print ("Vorher", variables_to_diff)
            print ("reduce {} with {}".format(e1.Lder(), e2.Lder()))
            print ("e1:", e1.Lterm())
            print ("e2:", e2.Lterm())
            e1._orig.show()
            e2._orig.show()
            print ("after derivation................")
            diff(e2._orig, *variables_to_diff).show()
            e3 = e1._orig - c * diff(e2._orig, *variables_to_diff)
            e3.show()
            dp =  Differential_Polynomial (e3, context)
            print ("e3:", dp.Lterm())            
            #for _ in dp._p:
            #    print (":3", _._d, _._coeff)            
            return dp
        else:
            print ("no reduction?")
            pass
    return e1

In [ ]:
var("x y")
z  = function ("z")(x,y)
ctx = Context((z,), (x,y))
print("*"*70)
e1 = diff(z,y)-x^2/y^2*diff(z,x)-(x-y)*z(x,y)/y^2
e1_dp = Differential_Polynomial (e1, ctx)
print (e1_dp)
print("*"*70)
f1 = diff(z, x) + z(x,y)/x
f1_dp = Differential_Polynomial (f1, ctx)
f1_dp.is_monic()
print (f1_dp)
print("*"*70)
f2 = diff(z, y) + z(x,y)/y
f2_dp = Differential_Polynomial (f2, ctx)
print (f2_dp)

In [ ]:
r1 = reduce (e1_dp, f1_dp, ctx)

In [ ]:
r1

In [ ]:
r1._orig

In [ ]:
r1._p

In [ ]:
f2_dp._p

In [ ]:
r2 = reduce (r1, f2_dp, ctx)

In [ ]:
r2

In [ ]:
def Lfunc(e):
    return e

In [ ]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [ ]:
def Lcm(e):
    return 0

In [ ]:
def CompleteSystem(S, context):
    return S

In [ ]:
def FindIntegrableConditions(S, context):
    return S

In [ ]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [ ]:
def Reorder (S, context):
    res=sorted(S, 
            key=functools.cmp_to_key(
                 lambda item1, item2: 
                                MatrixOrder.sorter (item1.Lder(), item2.Lder() , context._weight, 
                                                    tuple(context._dependent), 
                                                    tuple(context._independent)
                                                   )
                       )
                      )
    return res

In [ ]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 = Differential_Polynomial(diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y, ctx)
f1._orig

In [ ]:
f2=Differential_Polynomial(diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x,ctx)
f2._orig

In [ ]:
f3 = Differential_Polynomial(diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2 ,ctx)
f3._orig

In [ ]:
f4 = Differential_Polynomial(diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2), ctx)
f4._orig
f4._p

In [ ]:
f5 = Differential_Polynomial(diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/y**2, ctx)
f5._orig

In [ ]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        r = S[i+1:]
        if not r:
            S = Reorder(S, context)
            return S[:]            
        have_reduced = False
        checkidx = i+1
        for _r in r:
            print ("Try to reduce p{} with r{}".format(i, checkidx))
            res = reduce (p, _r, context)
            if res._orig != p._orig:
                print ("have reduced!")
                res._orig.show()
                have_reduced = True                
            S[i] = res
            checkidx += 1
        S = Reorder(S, context)
        print("This is the new System........")
        for _ in S:
            _._orig.show()
        if not have_reduced:
            i += 1        
        else:
            i = 0
            

In [ ]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    S = Reorder(S, context)
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [ ]:
checkS=JanetBasis([f3._orig,f1._orig,f5._orig, f2._orig,f4._orig], ctx)

In [ ]:
for i in (f1._orig, f2._orig,f3._orig, f4._orig,f5._orig):
    i.show()

In [ ]:
for _ in checkS:
    _._orig.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a